In [1]:
"|IMPORT PACKAGES|"
import numpy              as np
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Text, Span
from   bokeh.models.tools import HoverTool
from   bokeh.models       import Arrow, NormalHead, OpenHead, VeeHead
from   bokeh.transform    import dodge
from   datetime           import datetime as dt

In [2]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'BCRA'

data = pd.read_excel(IO, sheet_name = sheet, usecols="A, W, Y, AG", skiprows=1, nrows=1120, engine='openpyxl') # Be patient...

data = data.rename(columns={"Fechas"             :"WEEK"})
data = data.rename(columns={"Reservas (USD)"     :"RESERVAS"})
data = data.rename(columns={"Reservas Netas (I)" :"NETAS1"})
data = data.rename(columns={"Reservas Netas (II)":"NETAS2"})

data['RESERVAS'] = data['RESERVAS']/1000
data['NETAS1']   = data['NETAS1']  /1000
data['NETAS2']   = data['NETAS2']  /1000

In [3]:
"|CHECK DATA|"
data

,WEEK,RESERVAS,NETAS1,NETAS2
0,1998-01-07,18724.011701,18669.875734,11828.963738
1,1998-01-15,18347.342823,18311.080106,11468.969304
2,1998-01-23,17517.167207,17463.056002,10585.229304
3,1998-01-31,16911.677872,16900.855109,10067.626142
4,1998-02-07,17880.915523,17854.054498,10961.283143
...,...,...,...,...
1115,2021-03-31,39593.339023,28406.749916,1587.485503
1116,2021-04-07,39750.813700,28699.598321,1949.828862
1117,2021-04-15,40070.021095,29310.676053,2771.115524
1118,2021-04-23,40257.110636,29751.995318,3293.418115


In [4]:
"|BUILD PLOT|"

cds = ColumnDataSource(data)

#BUILD FIGURE
p = figure(title = "EL HUB ECONÓMICO | RESERVAS DEL BCRA (MILLONES DE US$)",
           x_axis_label = "",
           y_axis_label = "",
           x_axis_type  = "datetime",
           plot_height  = 400,
           plot_width   = 700)

p.toolbar_location = "right"
p.toolbar.autohide = True

#AXIS 1 (LEFT)
p1 = p.line('WEEK', 'RESERVAS', color='black', line_alpha=0.75, width=2, legend_label='Reservas brutas'  , muted_alpha=0.2, source=cds)
p2 = p.line('WEEK', 'NETAS1'  , color='blue' , line_alpha=0.75, width=2, legend_label='Reservas netas I' , muted_alpha=0.2, source=cds)
p3 = p.line('WEEK', 'NETAS2'  , color='red'  , line_alpha=0.75, width=2, legend_label='Reservas netas II', muted_alpha=0.2, source=cds)

p.add_tools(HoverTool(renderers=[p1], tooltips = [("Reservas Brutas"  , "@RESERVAS{$0,0}")], mode="vline"))
p.add_tools(HoverTool(renderers=[p2], tooltips = [("Reservas Netas I" , "@NETAS1{$0,0}")]  , mode="vline"))
p.add_tools(HoverTool(renderers=[p3], tooltips = [("Reservas Netas II", "@NETAS2{$0,0}")]  , mode="vline"))

#ADD NOTES
cepo1 = BoxAnnotation(left=dt(2011,10,23).timestamp()*1000, right=dt(2015,12,31).timestamp()*1000, line_color='black', fill_color='gray', fill_alpha=0.1)
cepo2 = BoxAnnotation(left=dt(2019, 9,30).timestamp()*1000,                                        line_color='black', fill_color='gray', fill_alpha=0.1)

x = (dt(2011,10,23).timestamp()*1000 + dt(2015,12,31).timestamp()*1000)/2
cepo1_text = Label(x=x, y=75000, text_font_size='12px', text_color='gray', text_align='center',
                  text='Cepo cambiario')

x = (dt(2019, 9,30).timestamp()*1000 + dt(2021, 2,28).timestamp()*1000)/2
cepo2_text = Label(x=x, y=75000, text_font_size='12px', text_color='gray', text_align='center',
                  text='Cepo cambiario')


p.add_layout(cepo1)
p.add_layout(cepo2)
p.add_layout(cepo1_text)
#p.add_layout(cepo2_text)

#ADD PRICE DUPLICATION MARKS
FMI_text = Label(x=dt(2005, 3,31).timestamp()*1000, y=45500, text_font_size='12px', text_color='gray', text_align='center',
                 text='Pago al FMI')
FMI_arrow = Arrow(end=VeeHead(line_color='gray', line_width=2, line_alpha=0.5, size=10, fill_color='gray', fill_alpha=0.75),
                  x_start=dt(2005, 3,31).timestamp()*1000, x_end=dt(2005,12,31).timestamp()*1000, y_start=45500, y_end=30000,
                  line_color='gray', line_alpha=0.5)

p.add_layout(FMI_text)
p.add_layout(FMI_arrow)

crisis1_text = Label(x=dt(2002, 2, 7).timestamp()*1000, y=25000, text_font_size='12px', text_color='gray', text_align='center',
                     text='$9.300M')
crisis1_arrow = Arrow(end=VeeHead(line_color='gray', line_width=2, line_alpha=0.5, size=10, fill_color='gray', fill_alpha=0.75),
                      x_start=dt(2002, 2, 7).timestamp()*1000, x_end=dt(2002,12,31).timestamp()*1000, y_start=25000, y_end=14000)

p.add_layout(crisis1_text)
p.add_layout(crisis1_arrow)

crisis2_text = Label(x=dt(2012, 6, 30).timestamp()*1000, y=52500, text_font_size='12px', text_color='gray', text_align='left',
                     text='$16.400M')
crisis2_arrow = Arrow(end=VeeHead(line_color='gray', line_width=2, line_alpha=0.5, size=10, fill_color='gray', fill_alpha=0.75),
                      x_start=dt(2012, 6,30).timestamp()*1000, x_end=dt(2014, 9,30).timestamp()*1000, y_start=52500, y_end=32000)

p.add_layout(crisis2_text)
p.add_layout(crisis2_arrow)

#LEGEND
h_axis = Span(location=0, dimension="width", line_color="black")
p.add_layout(h_axis)
p.xaxis[0].formatter = DatetimeTickFormatter(months='%b %Y')
p.yaxis[0].formatter = NumeralTickFormatter(format="$0,0")
p.legend.location     = "top_left"
p.legend.orientation  = "vertical"
p.legend.click_policy = "mute"
show(p)

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="reservas.png")

In [ ]:
"|# CREATE HTML FILE|"

output_file(filename="reservas.html", title="Reservas del BCRA (millones de US$)")
save(p)

In [5]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "reservas"))
with open("reservas.json", "w") as fp:
    json.dump(j, fp)